In [ ]:
import requests
from bs4 import BeautifulSoup

def get_news(keyword) : 
  headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}

  data = requests.get(f'https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query={keyword}',headers=headers)

  soup = BeautifulSoup(data.text, 'html.parser')

  lis = soup.select('#main_pack > section > div > div.group_news > ul > li')

  for li in lis : 
    a = li.select_one('a.news_tit')
    print(a.text, a['href'])

get_news('현대자동차')

In [ ]:
from openpyxl import Workbook

wb= Workbook()
sheet = wb.active

sheet['A1'] = '안녕하세요!'

wb.save("샘플파일.xlsx")
wb.close()

In [ ]:
import openpyxl
wb = openpyxl.load_workbook('샘플파일.xlsx')
sheet = wb['Sheet']

#sheet['A1'].value

rows = list(sheet.rows)[0:]
for row in rows : 
  print(row[0].value)


In [ ]:
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook
from datetime import datetime

def get_news(keyword) :
  wb= Workbook()
  sheet = wb.active
  
  headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}
  data = requests.get(f'https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query={keyword}',headers=headers)

  soup = BeautifulSoup(data.text, 'html.parser')

  lis = soup.select('#main_pack > section > div > div.group_news > ul > li')

  for li in lis : 
    a = li.select_one('a.news_tit')
    row = [a.text, a['href']]
    sheet.append(row)

  today = datetime.today().strftime("%Y-%m-%d")

  wb.save(f"news/{today}_{keyword}.xlsx")
  wb.close()

In [ ]:
keywords = ['삼성전자','LG이노텍','기업은행']

for keyword in keywords : 
  print(keyword)
  get_news(keyword)

In [ ]:
!zip -r /content/files.zip /content/news

In [ ]:
import os

path = '/content/news'
files = os.listdir(path)

for name in files:
  new_name = name.split('.')[0] + '(뉴스).xlsx'
  os.rename(f'/content/news/{name}', f'/content/news/{new_name}')

In [ ]:
import urllib.request

url = 'https://ssl.pstatic.net/imgfinance/chart/item/area/year3/005930.png'
urllib.request.urlretrieve(url, "LG디스플레이.png")

In [ ]:
#1주차 숙제
import openpyxl
import urllib.request

wb = openpyxl.load_workbook('관리종목.xlsx')
sheet = wb['종목']

rows = list(sheet.rows)[1:]

for row in rows : 
  print(row[0].value, row[1].value)
  url = f'https://ssl.pstatic.net/imgfinance/chart/item/area/year3/{row[1].value}.png'
  urllib.request.urlretrieve(url, f"image/{row[0].value}.png")

In [ ]:
# 2주차

In [ ]:
import pandas as pd
import numpy as np

data = {
    'name':['철수','영희','동희','영수'],
    'age':[15,12,20,35]
}

df = pd.DataFrame(data)

In [ ]:
doc = {
    'name':'세종',
    'age':23
}

df = df.append(doc, ignore_index=True)

In [ ]:
df['city'] = ['서울','부산','서울','부산','부산']

In [ ]:
cond = df['age'] < 20
df[cond]

In [ ]:
df.iloc[-1,0]

In [ ]:
df.sort_values(by='age',ascending=False)
df.sort_values(by='age',ascending=False).iloc[0,1]

In [ ]:
df['is_adult'] = np.where(df['age'] < 20, '청소년', '성인')

In [ ]:
df['age'].mean()
df['age'].min()
df['age'].describe()

In [ ]:
df[df['city']=='서울']['age'].mean()

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
df = pd.read_excel('종목데이터.xlsx')

cond = df['change_rate'] > 0
df[cond]

cond = df['per'] >0
df[cond]

df['Close'] = df['per'] * df['eps']
df['earning'] = df['marketcap'] / df['per']

del df['date']

cond = (df['pbr'] < 1) & (df['marketcap'] > 10000000000000) & (df['per'] < 20)
df = df[cond]

df.sort_values(by='marketcap', ascending=False)
df.describe()

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

In [ ]:
company = yf.Ticker('TSLA')
company.info

In [ ]:
name = company.info['shortName']
industry = company.info['industry']
marketcap = company.info['marketCap']
revenue = company.info['totalRevenue']

company.balance_sheet

company.balance_sheet.loc[['Cash']]

company.earnings

df = company.recommendations
cond = df['Firm'] == 'JP Morgan'
df[cond]

news = company.news
news[0]['title']

In [ ]:
def add_company(code):
  company = yf.Ticker(code)

  name = company.info['shortName']
  industry = company.info['industry']
  marketcap = company.info['marketCap']
  summary = company.info['longBusinessSummary']
  currentprice = company.info['currentPrice']
  targetprice = company.info['targetMeanPrice']
  per = company.info['trailingPE']
  eps = company.info['trailingEps']
  pbr = company.info['priceToBook']

  rev2021 = company.earnings.iloc[-1,0]
  rev2020 = company.earnings.iloc[-2,0]
  rev2019 = company.earnings.iloc[-3,0]

  ear2021 = company.earnings.iloc[-1,1]
  ear2020 = company.earnings.iloc[-2,1]
  ear2019 = company.earnings.iloc[-3,1]

  doc = {
      'name':name,
      'industry':industry,
      'marketcap':marketcap,
      'summary':summary,
      'currentprice':currentprice,
      'targetprice':targetprice,
      'per':per,
      'eps':eps,
      'pbr':pbr,
      'rev2021':rev2021,
      'rev2020':rev2020,
      'rev2019':rev2019,
      'ear2021':ear2021,
      'ear2020':ear2020,
      'ear2019':ear2019
  }

  return doc

In [ ]:
df = pd.DataFrame()

codes = ['AAPL','ABNB','BIDU','FB','GOOG','MSFT','TSLA','PYPL','NFLX','NVDA']

for code in codes :
  print(code)
  try :
    row = add_company(code)
    df = df.append(row, ignore_index=True)
  except :
    continue

df

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
df.sort_values(by = 'eps', ascending = False)

In [ ]:
cond = df['per'] < 30
df[cond].sort_values(by='per', ascending=False)

In [ ]:
new_df = df[['name','currentprice','targetprice']].copy()

new_df['gap'] = new_df['targetprice'] / new_df['currentprice'] - 1

new_df.sort_values(by='gap', ascending=False)


In [ ]:
new_df = df[['name', 'ear2021','ear2020','ear2019']].copy()

cond = (new_df['ear2021'] > new_df['ear2020']) & (new_df['ear2020'] > new_df['ear2019'])

new_df['is_target'] = np.where(cond, 'O', 'X')

new_df

In [ ]:
#2주차 숙제
df= pd.DataFrame()
company = yf.Ticker('TSLA')
name = company.info['shortName']
a2021 = company.balance_sheet.loc[['Cash']].iloc[-1,0]
a2020 = company.balance_sheet.loc[['Cash']].iloc[-1,1]

row = {
    'name':name,
    '2021': a2021,
    '2020': a2020,
}

df = df.append(row, ignore_index=True)
df['diff'] = df['2021'] - df['2020']
df

In [ ]:
#Dart Open-Api key : fc748156cc672cb16311817e362a748e940e9960

In [ ]:
#3주차

In [ ]:
!pip install dart-fss

In [ ]:
import dart_fss as dart_fss
import pandas as pd

api_key = 'fc748156cc672cb16311817e362a748e940e9960'
dart_fss.set_api_key(api_key=api_key)

In [ ]:
all = dart_fss.api.filings.get_corp_code()

In [ ]:
df = pd.DataFrame(all)

df_listed = df[df['stock_code'].notnull()]
df_non_listed = df[df['stock_code'].isnull()]

In [ ]:
corp_code = df_listed[df_listed['corp_name']=='카카오'].iloc[0,0]
dart_fss.api.filings.get_corp_info(corp_code)

In [ ]:
corp_code = df_listed[df_listed['corp_name']=='카카오'].iloc[0,0]
data = dart_fss.api.info.unrst_exctv_mendng_sttus(corp_code,'2021', '11011')
# 보고서 : dart_fss.api.info.unrst_exctv_mendng_sttus
# 증자 : dart_fss.api.info.irds_sttus
# 배당 : dart_fss.api.info.alot_matter
# 최대주주 : dart_fss.api.info.hyslr_sttus
# 직원 : dart_fss.api.info.emp_sttus
# 5억이상 상위5인 : dart_fss.api.info.indvdl_by_pay(
pd.DataFrame(data['list'])

In [ ]:
# 연봉킹 뽑기
def get_salary(name):
  corp_code = df_listed[df_listed['corp_name'] == name].iloc[0,0]
  data = dart_fss.api.info.indvdl_by_pay(corp_code,'2021', '11011')

  df = pd.DataFrame(data['list'])
  df = df[['corp_name', 'nm', 'ofcps', 'mendng_totamt']]
  df.columns = ['기업명', '이름', '역할', '보수']
  df['보수'] = pd.to_numeric(df['보수'].str.replace(',',""))
  #df.dtypes 자료형 확인
  df.sort_values(by='보수', ascending=False)

  return df

In [ ]:
names = ['삼성전자','LG에너지솔루션','SK하이닉스','NAVER','삼성바이오로직스','삼성전자우','카카오','삼성SDI','현대차','LG화학','기아','POSCO홀딩스','KB금융','카카오뱅크','셀트리온','신한지주','삼성물산','현대모비스','SK이노베이션','LG전자','카카오페이','SK','한국전력','크래프톤','하나금융지주','LG생활건강','HMM','삼성생명','하이브','두산중공업','SK텔레콤','삼성전기','SK바이오사이언스','LG','S-Oil','고려아연','KT&G','우리금융지주','대한항공','삼성에스디에스','현대중공업','엔씨소프트','삼성화재','아모레퍼시픽','KT','포스코케미칼','넷마블','SK아이이테크놀로지','LG이노텍','기업은행']
dfs = []
for name in names :
  try :
    df = get_salary(name)
    dfs.append(df)
  except : 
    print(f'error - {name}')

df_result = pd.concat(dfs)
df_result.sort_values(by='보수', ascending=False).head(30)

In [ ]:
# 최대주주 지분율 변화 뽑기
def get_shareholders(corp_code) : 
  data = dart_fss.api.info.hyslr_sttus(corp_code,'2021', '11011')

  df = pd.DataFrame(data['list'])
  df = df[['corp_name', 'nm', 'relate', 'bsis_posesn_stock_qota_rt', 'trmend_posesn_stock_qota_rt', 'rm']]
  df.columns = ['기업명', '이름', '관계', '기초지분율', '기말지분율', '비고']

  df = df[df['관계'].notnull()]

  df['기초지분율'] = pd.to_numeric(df['기초지분율'])
  df['기말지분율'] = pd.to_numeric(df['기말지분율'])
  return df.sort_values(by='기초지분율', ascending = False).head(3)

In [ ]:
corp_codes = list(df_listed.sample(10)['corp_code'])

dfs = []
for corp_code in corp_codes : 
  try : 
    df = get_shareholders(corp_code)
    dfs.append(df)
  except : 
    print(f'error - {corp_code}')

df_result = pd.concat(dfs)
df_result['증감'] = df_result['기말지분율'] - df_result['기초지분율']

df_result.sort_values(by='증감', ascending=False)

In [ ]:
#돈 많이 번 회사 찾기
def get_profit(name):
  corp_code = df_listed[df_listed['corp_name'] == name].iloc[0,0]
  data = dart_fss.api.finance.fnltt_singl_acnt(corp_code, '2021', '11011')
  df = pd.DataFrame(data['list'])

  cond = (df['fs_div'] == 'CFS') & (df['account_nm'] == '이익잉여금')
  df = df[cond]

  df['name'] = name

  df = df[['name', 'thstrm_amount','frmtrm_amount']]
  df.columns = ['기업명','당기', '전기']
  df['당기'] = pd.to_numeric(df['당기'].str.replace(',',''))
  df['전기'] = pd.to_numeric(df['전기'].str.replace(',',''))

  df['증감'] = df['당기'] - df['전기']
  df['증감률'] = df['증감'] / abs(df['전기'])
  
  return df

In [ ]:
names = list(df_listed.sample(10)['corp_name'])

dfs = []
for name in names :
  try : 
    df = get_profit(name)
    dfs.append(df)
  except :
    print(f'error - {name}')

df_result = pd.concat(dfs)
df_result.sort_values(by='증감률', ascending=False)

In [ ]:
def get_earning(name):
  corp_code = df_non_listed[df_non_listed['corp_name'] == name].iloc[0,0]
  data = dart_fss.api.info.alot_matter(corp_code, '2021', '11011')
  df = pd.DataFrame(data['list'])

  df = df[df['se'] == '(연결)당기순이익(백만원)']
  df = df[['corp_name','thstrm','frmtrm','lwfr']]

  df.columns = ['기업명', '2021', '2020', '2019']
  df['2021'] = pd.to_numeric(df['2021'].str.replace(',',''))
  df['2020'] = pd.to_numeric(df['2020'].str.replace(',',''))
  df['2019'] = pd.to_numeric(df['2019'].str.replace(',',''))
  return df

In [ ]:
names = list(df_non_listed.sample(10)['corp_name'])

dfs = []
for name in names :
  try :
    df = get_earning(name)
    dfs.append(df)
  except : 
    print(f'error - {name}')

df_result = pd.concat(dfs)
df_result
                      

In [ ]:
#3주차 숙제
def get_salary(name) : 
  corp_code = df_listed[df_listed['corp_name'] == name].iloc[0,0]
  data = dart_fss.api.info.emp_sttus(corp_code, '2021', '11011')

  df = pd.DataFrame(data['list'])
  df = df[['corp_name', 'sexdstn', 'jan_salary_am']]

  df_result = pd.DataFrame()
  doc = {
        '기업명' : name,
        '연봉(남)' : df[df['sexdstn'] == '남'].iloc[-1,-1],
        '연봉(여)' : df[df['sexdstn'] == '여'].iloc[-1,-1]
  }
  df_result = df_result.append(doc, ignore_index=True)

  df_result['연봉(남)'] = pd.to_numeric(df_result['연봉(남)'].str.replace(',',''))
  df_result['연봉(여)'] = pd.to_numeric(df_result['연봉(여)'].str.replace(',',''))
  df_result['차이(남-여)'] = df_result['연봉(남)'] - df_result['연봉(여)']
  df_result['평균'] = (df_result['연봉(남)'] + df_result['연봉(여)'])/2
  return df_result

names = list(df_listed.sample(10)['corp_name'])

dfs = []
for name in names : 
  try :
    df = get_salary(name)
    dfs.append(df)
  except : 
    print(f'error-{name}')

df_result = pd.concat(dfs)
df_result.sort_values(by='차이(남-여)', ascending=True)

In [ ]:
#4주차

In [ ]:
!pip install yfinance pandas-datareader finance-datareader

In [ ]:
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()
import numpy as np
import pandas as pd
import FinanceDataReader as fdr

In [ ]:
df_1 = fdr.DataReader('005930','2018')
df_2 = fdr.DataReader('066570','2018')

df = pd.DataFrame()
df['Samsung'] = df_1['Change']
df['LG'] = df_2['Change']

df.tail(100).plot(figsize=[15,8], grid=True)

In [ ]:
df = fdr.DataReader('005930','2018')

df = df[['Close']]

df['ma'] = df.rolling(3).mean()

df

In [ ]:
df = fdr.DataReader('005930','2018')

df = df[['Close']]

df['ma'] = df.rolling(3).mean().shift(1)

df['action'] = np.where(df['Close'] > df['ma'], 'buy', 'sell')

df

In [ ]:
df = fdr.DataReader('005930','2018')

df = df[['Close']]

df['ma'] = df.rolling(3).mean().shift(1)

df['action'] = np.where(df['Close'] > df['ma'], 'buy', 'sell')

cond1 = (df['action']=='buy') & (df['action'].shift(1) == 'sell')
cond2 = (df['action']=='sell') & (df['action'].shift(1) == 'buy')

df_buy = df[cond1]
df_sell = df[cond2]

df_result = pd.concat([df_buy, df_sell], axis=1)

df_result.head(30)

In [ ]:
# 이동평균선 몇일선과 수익률 누적 구하기
def get_return(code, n):
  df = fdr.DataReader(code,'2018')

  df = df[['Close']].copy()

  df['ma'] = df.rolling(n).mean().shift(1)

  df['action'] = np.where(df['Close'] > df['ma'], 'buy', 'sell')

  cond1 = (df['action']=='buy') & (df['action'].shift(1) == 'sell')
  cond2 = (df['action']=='sell') & (df['action'].shift(1) == 'buy')

  df_buy = df[cond1].reset_index()
  df_buy.columns = ['날짜', '종가(buy)', '이평값', '액션']
  df_sell = df[cond2].reset_index()
  df_sell.columns = ['날짜', '종가(sell)', '이평값', '액션']
  df_result = pd.concat([df_buy, df_sell], axis=1)

  df_result['수익률'] = df_result['종가(sell)'] / df_result['종가(buy)']
  return df_result[['수익률']].cumprod().iloc[-2,-1] - 1

In [ ]:
get_return('005930',448)

In [ ]:
def get_return_sl(code, short, long) : 
  df = fdr.DataReader(code,'2018')

  df = df[['Close']].copy()

  df['ma1'] = df['Close'].rolling(short).mean().shift(1)
  df['ma2'] = df['Close'].rolling(long).mean().shift(1)

  df['action'] = np.where(df['ma1'] > df['ma2'], 'buy', 'sell')

  df.iloc[-1,-1] = 'sell'

  cond1 = (df['action']=='buy') & (df['action'].shift(1) == 'sell')
  cond2 = (df['action']=='sell') & (df['action'].shift(1) == 'buy')

  df_buy = df[cond1].reset_index()
  df_buy.columns = ['날짜', '종가(buy)', '이평값1', '이평값2', '액션']
  df_sell = df[cond2].reset_index()
  df_sell.columns = ['날짜', '종가(sell)', '이평값1', '이평값2', '액션']

  df_result = pd.concat([df_buy, df_sell], axis=1)

  df_result['수익률'] = df_result['종가(sell)'] / df_result['종가(buy)']
  df_final = (df_result[['수익률']].cumprod().tail(1) -1) * 100
  df_final['단기'] = short
  df_final['장기'] = long

  return df_final

In [ ]:
get_return_sl('005930',3,30)

In [ ]:
#4주차 숙제
def get_return_sl(code, short, long) : 
  df = fdr.DataReader(code,'2018')

  df = df[['Close']].copy()

  df['ma1'] = df['Close'].rolling(short).mean().shift(1)
  df['ma2'] = df['Close'].rolling(long).mean().shift(1)

  df['action'] = np.where(df['ma1'] > df['ma2'], 'buy', 'sell')

  df.iloc[-1,-1] = 'sell'

  cond1 = (df['action']=='buy') & (df['action'].shift(1) == 'sell')
  cond2 = (df['action']=='sell') & (df['action'].shift(1) == 'buy')

  df_buy = df[cond1].reset_index()
  df_buy.columns = ['날짜', '종가(buy)', '이평값1', '이평값2', '액션']
  df_sell = df[cond2].reset_index()
  df_sell.columns = ['날짜', '종가(sell)', '이평값1', '이평값2', '액션']

  df_result = pd.concat([df_buy, df_sell], axis=1)

  df_result['수익률'] = df_result['종가(sell)'] / df_result['종가(buy)']
  df_final = (df_result[['수익률']].cumprod().tail(1) -1) * 100
  df_final['단기'] = short
  df_final['장기'] = long

  return df_final

dfs = []
for short in range(3,11):
  for long in range(30,61):
      df = get_return_sl('005930',short, long)
      dfs.append(df)

df_result = pd.concat(dfs)
df_result.sort_values(by='수익률', ascending = False).head(1)

In [ ]:
#5주차

In [ ]:
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()
import numpy as np
import pandas as pd
import FinanceDataReader as fdr

In [ ]:
def get_return(code, k) : 

  df = fdr.DataReader(code,'2018')

  df['buy_at'] = (df['High'].shift(1) - df['Low'].shift(1)) * k + df['Open']

  df['is_buy'] = np.where(df['High'] > df['buy_at'], 'buy', '')

  df['sell_at'] = df['Open'].shift(-1)

  df = df[df['is_buy'] == 'buy']

  df['return'] = df['sell_at'] / df['buy_at']

  return df[['return']].cumprod().iloc[-1,-1] - 1

In [ ]:
get_return('005930',0.4)

In [ ]:
df = pd.DataFrame()

for k in np.arange(0.4,0.6,0.01):
  doc = {
      'k' : k,
      'return' : get_return('005930',k)
  }
  df = df.append(doc, ignore_index=True)

df.sort_values(by='return', ascending=False)

In [ ]:
#월요일에 사서 금요일에 팔기
def get_return_mf(code) : 
  df = fdr.DataReader(code,'2020')

  df = df.reset_index()

  df = df[['Date','Open']]

  df['Day'] = pd.to_datetime(df['Date']).dt.day_name()

  cond = (df['Day'] =='Monday') | (df['Day'] =='Friday')

  df = df[cond]



  if df.iloc[0,2] == 'Friday' :
    df = df.iloc[1:,:]

  if df.iloc[-1,2] == 'Monday' :
    df = df.iloc[:-1,:]

  df['Open_Friday'] = df['Open'].shift(-1)

  df = df[df['Day'] == 'Monday'].copy()

  df = df[['Open', 'Open_Friday']]

  df.columns = ['buy_at', 'sell_at']

  df['return'] = df['sell_at'] / df['buy_at']

  df = df[df['buy_at']!=0]

  return df[['return']].cumprod().iloc[-1,-1] -1

In [ ]:
get_return_mf('005930')

In [ ]:
#최적의 종목찾기

In [ ]:
!pip install dart-fss

In [ ]:
import dart_fss as dart_fss
import pandas as pd

api_key = 'fc748156cc672cb16311817e362a748e940e9960'
dart_fss.set_api_key(api_key=api_key)

all = dart_fss.api.filings.get_corp_code()

df = pd.DataFrame(all)

df_listed = df[df['stock_code'].notnull()]

In [ ]:
#print(row[1], row[2]) 으로 코드랑 이름 가져올 수 있음
  
df = pd.DataFrame()
for row in df_listed.sample(50)[['stock_code','corp_name']].itertuples():
  try : 
    doc = {
        'name' : row[2],
        'return' : get_return_mf(row[1])
    }
    df = df.append(doc, ignore_index=True)
  except : 
    print(f'error-f{row[2]}')

df.sort_values(by = 'return', ascending=False)